In [1]:
import tensorflow.compat.v1 as tf 
tf.disable_v2_behavior()
from tensorflow.examples.tutorials.mnist import input_data

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
tf.set_random_seed(777)

# 데이터 가져오기
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
learning_late = 0.001
training_epochs = 15
batch_size = 100

In [4]:
class Model:
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()
        
    def _build_net(self):
        with tf.variable_scope(self.name):
            # keep
            self.training = tf.placeholder(tf.bool)
            # input place holders
            self.X = tf.placeholder(tf.float32, [None, 784]) # 784 = 28*28
            
            # image 28*28, level 1, input layer 
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])
            
            # Convolutional Layer #1
            conv1 = tf.layers.conv2d(inputs=X_img, filters=32, kernel_size=[3,3], padding="SAME",activation=tf.nn.relu)
            # Pooling Layer #1
            pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], padding="SAME", strides=2)
            dropout1 = tf.layers.dropout(inputs=pool1, rate =0.3, training=self.training)            
            
            # Convolutional Layer #2
            conv2 = tf.layers.conv2d(inputs=dropout1, filters=64, kernel_size=[3,3], padding="SAME", activation=tf.nn.relu)
            # Pooling Layer #2
            pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2], padding='SAME', strides=2)
            dropout2 = tf.layers.dropout(inputs=pool2, rate =0.3, training=self.training)
            
            # Convolutional Layer #2
            conv3 = tf.layers.conv2d(inputs=dropout2, filters=128, kernel_size=[3,3], padding="SAME", activation=tf.nn.relu)
            # Pooling Layer #2
            pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2,2], padding='SAME', strides=2)
            dropout3 = tf.layers.dropout(inputs=pool3, rate =0.3, training=self.training)
            
            flat = tf.reshape(dropout3, [-1, 128*4*4])
            dense4 = tf.layers.dense(inputs=flat, units=625, activation=tf.nn.relu)
            dropout4 = tf.layers.dropout(inputs= dense4, rate=0.5, training=self.training )
            
            self.logits = tf.layers.dense(inputs=dropout4, units = 10)
        
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                                    logits=self.logits, labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_late).minimize(self.cost)
        
    def predict (self, x_test, training=False):
        return self.sess.run(self.logits, feed_dict = {self.X: x_test, self.training: training})
    
    def get_accuracy(self, x_test, y_test, training=False):
        return self.sess.run(self.accuracy, feed_dict={self.X:x_test, self.Y:y_test, self.training: training})
    
    def train(self, x_data, y_data, training=False):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.X:x_data, self.Y:y_data, self.training: training})           
            
            

In [5]:
sess = tf.Session()
m1 = Model(sess, "m1")

sess.run(tf.global_variables_initializer())
print('Learning Started')

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _ = m1.train(batch_xs, batch_ys)
        avg_cost += c /total_batch
        
    print('Epoch:', '%04d' %(epoch +1), 'cost=', '{:.9f}'.format(avg_cost))
    
print('Learning Finished!')

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Learning Started
Epoch: 0001 cost= 0.173675472
Epoch: 0002 cost= 0.043537198
Epoch: 0003 cost= 0.029986789
Epoch: 0004 cost= 0.021207255
Epoch: 0005 cost= 0.016978092
Epoch: 0006 cost= 0.015298568
Epoch: 0007 cost= 0.012380294
Epoch: 0008 cost= 0.010818490
Epoch: 0009 cost= 0.008507110
Epoch: 0010 cost= 0.007230140
Epoch: 0011 cost= 0.008286827
Epoch: 0012 cost= 0.007008583
Epoch: 0013 cost= 0.005750240
Epoch: 0014 cost= 0.004972284
Epoch: 0015 cost= 0.002783606
Learning Finished!
